# Prep dataset

In [ ]:
import pandas as pd
import numpy as np
import os
import scipy
import scipy.stats

from google.colab import drive
drive.mount('/content/drive')
data_foloder = 'here'
result = 'here'

baseline = pd.read_csv(data_foloder + 'here.csv')
incoh10 = pd.read_csv(data_foloder + 'here.csv')
incoh20 = pd.read_csv(data_foloder + 'here.csv')
incoh50 = pd.read_csv(data_foloder + 'here.csv')
ineff10 = pd.read_csv(data_foloder + 'here.csv')
ineff20 = pd.read_csv(data_foloder + 'here.csv')
ineff50 = pd.read_csv(data_foloder + 'here.csv')

baseline.head()

In [2]:
os.getcwd()

'/content'

# Install lib and dependencies

In [3]:
# Use colab because: M1 keeps having issues with tf; 
# faster with TPU; all deidentified; don't need API key.
# Install transformers from hugging face

!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.6 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 


In [4]:
# T5 uses SentencePiece tokenizer, 
# which is implemented in C and is opaque to Python.
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.5 MB/s 


In [5]:
# Using t5-small model and tokenizer

from transformers import T5Tokenizer, TFT5Model

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5Model.from_pretrained('t5-small')

# tf: tensorflow
inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="tf") # Batch size 1
outputs = model(inputs, decoder_input_ids=inputs)

# The last hidden-state is the first element of the output tuple
last_hidden_states = outputs[0]  # becauses it takes advantage of the whole context

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5Model.

All the layers of TFT5Model were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.


In [6]:
inputs

<tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[8774,    6,   82, 1782,   19, 5295,    1]], dtype=int32)>

In [7]:
last_hidden_states

<tf.Tensor: shape=(1, 7, 512), dtype=float32, numpy=
array([[[ 1.43691644e-01,  1.43860221e-01, -4.79405001e-03, ...,
         -2.31729019e-02,  1.08577289e-04,  1.61265135e-01],
        [ 1.19308271e-01,  3.92194912e-02,  1.62899643e-02, ...,
         -7.39525035e-02,  1.65885955e-04,  2.14623004e-01],
        [ 1.86311349e-01,  9.69478935e-02, -5.15518859e-02, ...,
         -2.02436835e-01,  2.43982286e-04, -4.03858960e-01],
        ...,
        [-1.02484174e-01,  1.72806203e-01, -9.05705914e-02, ...,
         -1.09222971e-01,  1.99182105e-04, -9.81829464e-02],
        [ 4.98955436e-02,  3.99321705e-01, -7.35218301e-02, ...,
         -4.26821746e-02, -1.41399854e-04, -1.92345567e-02],
        [ 9.09105688e-02,  3.13064575e-01, -2.96856426e-02, ...,
         -4.46503200e-02,  3.76074604e-04,  4.69613001e-02]]],
      dtype=float32)>

In [8]:
inputs = tokenizer.encode("Hello, my dog is very cute", return_tensors="tf") # Batch size 1
inputs # tbd if we want to include '1' and '6' [spe special tokens; punctuations] in similarity calculation [ask Joao and Reno]

<tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[8774,    6,   82, 1782,   19,  182, 5295,    1]], dtype=int32)>

# Get contextualized word embeddings

In [ ]:
# define a function to get contextualized embeddings
def get_last_hidden_state(sent):
  inputs = tokenizer.encode(sent, return_tensors="tf")  # Batch size 1
  outputs = model(inputs, decoder_input_ids=inputs)
  # The last hidden-state is the first element of the output tuple (c.f. Raffel et al 2020)
  last_hidden_states = outputs[0]  
  return last_hidden_states

# call the function 
# apply it to the dataframe
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  df["t5_word_embed"] = ''
  df["t5_word_embed"] = df['content'].apply(lambda x: get_last_hidden_state(x)) 
  df.to_csv(result + str(temp) + '.csv')

df.head()

In [12]:
# double check if it's in good shape
df['t5_word_embed'][0].shape # 231 tokens, each token has 512 dimensions [for T5]

TensorShape([1, 529, 512])

# Stats and similarities functions

In [ ]:
# stats ignoring nan, apply to all LMs 
from numpy import nanmedian

import scipy
def iqr(x):
  return scipy.stats.iqr(np.array(x), nan_policy='omit')

from numpy import nanquantile
def q5(x):
    return np.nanquantile(np.array(x), 0.05)

def q95(x):
    return np.nanquantile(np.array(x), 0.95)

In [ ]:
# cosine_similarity, apply to all LMs
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# MV 5/10

In [13]:
# Average semantic similarity of each word in 5- or 10- words window

def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
# n: How many elements each
# list should have
test = ['Alex','broke','the','vase','accidentally','.','But','Kai','did','it','on','purpose','.']
divide_chunks(test,5)
chopped = list(divide_chunks(test,5))
print(chopped)

[['Alex', 'broke', 'the', 'vase', 'accidentally'], ['.', 'But', 'Kai', 'did', 'it'], ['on', 'purpose', '.']]


In [24]:
def combinations(lst): # get w1, w2 combinations
    # input: a list of <= 5 tokens
    cmb = []
    rightside = lst[:] # initialize a list
    for wid, w1 in enumerate(lst): # each token gets a chance to be w1
        rightside = lst[wid:] # dynamically chop off w1 from the rest of the list
        while rightside: # loop until the rest of the list is empty
            w2 = rightside.pop(0) # stack up w2
            if w2 != w1: # get rid of ['Alex', 'Alex']
                cmb.append([w1, w2])  
    return cmb

testing = ['Alex', 'broke', 'the', 'vase', 'accidentally']
test_result = combinations(testing)
print(test_result)

[['Alex', 'broke'], ['Alex', 'the'], ['Alex', 'vase'], ['Alex', 'accidentally'], ['broke', 'the'], ['broke', 'vase'], ['broke', 'accidentally'], ['the', 'vase'], ['the', 'accidentally'], ['vase', 'accidentally']]


In [ ]:
# prep dataframe
# append columns to the embeddings df

mvs=['5', '10']
stats = ['_median', '_iqr', '_q5', '_q95'] 
for df in dfs:
    # create new empty columns
    for mv in mvs:
        for stat in stats:
            cur = 't5_word_mv' + mv + stat
            df[cur] = ''
df.head()

In [55]:
df['t5_word_embed'][0]

<tf.Tensor: shape=(1, 601, 512), dtype=float32, numpy=
array([[[-3.10553033e-02, -4.21899930e-03, -6.21268898e-02, ...,
         -9.61099789e-02,  3.67578468e-04,  1.27705187e-02],
        [ 1.78670580e-03, -5.48904948e-02, -8.62718299e-02, ...,
         -1.02402024e-01,  4.00742167e-04,  2.77379677e-02],
        [-6.48797005e-02,  1.07468605e-01, -2.83668507e-02, ...,
         -7.60212839e-02,  3.04235349e-04, -1.21393576e-01],
        ...,
        [ 8.11153501e-02,  2.59681195e-01, -1.54238269e-01, ...,
          9.70805958e-02, -2.47135235e-04, -2.73098737e-01],
        [ 1.07268795e-01,  2.65998274e-01, -1.26439348e-01, ...,
          8.88630673e-02, -2.35899861e-04, -1.69679955e-01],
        [ 6.79381862e-02,  3.10879678e-01, -1.08346254e-01, ...,
          9.01298374e-02, -2.72594480e-04, -1.66191489e-01]]],
      dtype=float32)>

In [30]:
df['t5_word_embed'][0][0]

<tf.Tensor: shape=(601, 512), dtype=float32, numpy=
array([[-3.10553033e-02, -4.21899930e-03, -6.21268898e-02, ...,
        -9.61099789e-02,  3.67578468e-04,  1.27705187e-02],
       [ 1.78670580e-03, -5.48904948e-02, -8.62718299e-02, ...,
        -1.02402024e-01,  4.00742167e-04,  2.77379677e-02],
       [-6.48797005e-02,  1.07468605e-01, -2.83668507e-02, ...,
        -7.60212839e-02,  3.04235349e-04, -1.21393576e-01],
       ...,
       [ 8.11153501e-02,  2.59681195e-01, -1.54238269e-01, ...,
         9.70805958e-02, -2.47135235e-04, -2.73098737e-01],
       [ 1.07268795e-01,  2.65998274e-01, -1.26439348e-01, ...,
         8.88630673e-02, -2.35899861e-04, -1.69679955e-01],
       [ 6.79381862e-02,  3.10879678e-01, -1.08346254e-01, ...,
         9.01298374e-02, -2.72594480e-04, -1.66191489e-01]], dtype=float32)>

In [ ]:
# combinations 'alex broke the vase': 
# alex broke, alex the, alex vase; broke the, broke vase; the vase
# Parola et al is inspired by Pauselli et al (p76): 
# ''Coherence is the average similarity of each word to each of the other
# words in the list, regardless of order or proximity. ''
#w2 = word_embed[word_id+1] # adjacent neighbour only
temp_file = -1
for df in dfs:
    temp_file += 1
    print(temp_file)
    for mv in mvs:
        # print progress
        cur = 't5_word_mv' + mv
        print('current: ', cur)
        df[cur + '_similarity'] = '' # save the cosine similarities; all stats are derived from there

        # loop over each response
        for i in df.index:
            if type(df['t5_word_embed'][i]) != float: 
                # chop 1 big response sequence into 5/10-token chunks
                word_embed_chunk = list(divide_chunks(df['t5_word_embed'][i][0], int(mv))) # add [0] for T5 because it's 3Dtensor
                chunk_temp_collection = [] 
                # loop over each 5/10 chunk in the response
                for chunck_id, word_embed in enumerate(word_embed_chunk):
                    temp_collection = []
                    # calculate average similarities for that chunk (5 or 10 window)                   
                    cmbs = combinations(word_embed.numpy().tolist()) # apply function # add .numpy().tolist() for T5, change 2Dtensor to 2Darray to list of lists
                    for cmb in cmbs:
                        w1 = cmb[0]
                        w2 = cmb[1]
                        temp = cosine_similarity(w1, w2)
                        temp_collection.append(temp)
                    temp_sim = np.nanmean(temp_collection)
                    chunk_temp_collection.append(temp_sim) # incrementally append similarity mean to the list 

            # get a list of similarity means for that response, 
            # its len is the number of chunks that the response can be chopped into
            df[cur + '_similarity'][i] = chunk_temp_collection # similarity mv 5 or 10; store it for later reference/stats

            # add other stats here
            df[cur + '_median'][i] = np.nanmedian(chunk_temp_collection)
            df[cur + '_q5'][i] = q5(chunk_temp_collection)
            df[cur + '_q95'][i] = q95(chunk_temp_collection)
            df[cur + '_iqr'][i] = iqr(chunk_temp_collection)
    df.to_csv(result + str(temp_file) + '.csv')
df.head()

In [49]:
df.columns

Index(['grid', 'content', 'n_words', 't5_word_embed', 't5_word_mv5_median',
       't5_word_mv5_iqr', 't5_word_mv5_q5', 't5_word_mv5_q95',
       't5_word_mv10_median', 't5_word_mv10_iqr', 't5_word_mv10_q5',
       't5_word_mv10_q95', 't5_word_mv5_similarity',
       't5_word_mv10_similarity'],
      dtype='object')

# K1:10

In [ ]:
import ast # a module that evaluates mathematical expressions and statements

ks=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
stats = ['_median', '_iqr', '_q5', '_q95']
for df in dfs:
    # create new empty columns
    for k in ks:
        for stat in stats:
            cur = 't5_word_k' + k + stat
            df[cur] = ''
df.head()

In [ ]:
temp_file = -1
ks=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

for df in dfs:
    temp_file += 1
    # loop through each k
    for k in ks:
        cur = 't5_word_k' + k 
        print('Coherence k ', k, 'temp_file: ', temp_file) # progress
        df[cur + '_similarity'] = ''
        # loop through each individual's response 
        for i in df.index:
            if type(df['t5_word_embed'][i]) != float:
                temp = []
                # calcuate similarity of word pairs at k inter-token distance
                for id,v in enumerate(df['t5_word_embed'][i][0].numpy().tolist()): # add [0].numpy().tolist() for T5 because it's 3Dtensor
                    w1 = v
                    try:
                        w2 = df['t5_word_embed'][i][0].numpy().tolist()[id + int(k)] # same here, for T5
                    except IndexError:
                        continue
                    sim = cosine_similarity(w1, w2)
                    temp.append(sim) # a list of similarity scores for that response

                # intermediate df, save 
                df[cur + '_similarity'][i] = temp
                df[cur + '_iqr'][i] = iqr(temp) # add other stats here
                df[cur + '_median'][i] = np.nanmedian(temp)
                df[cur + '_q5'][i] = q5(temp)
                df[cur + '_q95'][i] = q95(temp)
    df.to_csv(result + str(temp_file) + '.csv')
df.head()

In [60]:
df.columns 

Index(['grid', 'content', 'n_words', 't5_word_embed', 't5_word_mv5_median',
       't5_word_mv5_iqr', 't5_word_mv5_q5', 't5_word_mv5_q95',
       't5_word_mv10_median', 't5_word_mv10_iqr', 't5_word_mv10_q5',
       't5_word_mv10_q95', 't5_word_mv5_similarity', 't5_word_mv10_similarity',
       't5_word_k1_median', 't5_word_k1_iqr', 't5_word_k1_q5',
       't5_word_k1_q95', 't5_word_k2_median', 't5_word_k2_iqr',
       't5_word_k2_q5', 't5_word_k2_q95', 't5_word_k3_median',
       't5_word_k3_iqr', 't5_word_k3_q5', 't5_word_k3_q95',
       't5_word_k4_median', 't5_word_k4_iqr', 't5_word_k4_q5',
       't5_word_k4_q95', 't5_word_k5_median', 't5_word_k5_iqr',
       't5_word_k5_q5', 't5_word_k5_q95', 't5_word_k6_median',
       't5_word_k6_iqr', 't5_word_k6_q5', 't5_word_k6_q95',
       't5_word_k7_median', 't5_word_k7_iqr', 't5_word_k7_q5',
       't5_word_k7_q95', 't5_word_k8_median', 't5_word_k8_iqr',
       't5_word_k8_q5', 't5_word_k8_q95', 't5_word_k9_median',
       't5_word_k9_iqr